# Comparing performance of belief maximization algorithm with and without observabilities

In [25]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [26]:
import sys
sys.path.append('../')

In [27]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout

In [28]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [29]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [30]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)

<module 'source.players.belief_max' from '..\\source\\players\\belief_max.py'>

In [31]:
time_horizon = 1000
T = 5
#targets = util.gen_targets(T, low=1, high=10)
targets = [5, 3, 8, 4, 9]
values = tuple((v, v) for v in targets)
#obs = gen_observabilities_correlated_with_values(values)
obs = {0: 0.95, 1: 0.95, 2: 0.75, 3: 0.95, 4: 0.95}
obs_vect = [obs.get(i) for i in range(len(targets))]
g = game.Game(values, time_horizon)
g.attackers = [1]
g.defenders = [0]
a = [attackers.StackelbergAttacker(g, 1)]
d = [bm.FB(g, 0)]
p = [attackers.ObservingStackelbergAttacker(g, 1), attackers.StackelbergAttacker(g, 1)]
g.set_players(d, a, p)

go1 = game.GameWithObservabilities(values, time_horizon)
go1.attackers = [1]
go1.defenders = [0]
a1 = [attackers.StackelbergAttacker(go1, 1)]
d1 = [bm.FB(go1, 0)]
p1 = [attackers.ObservingStackelbergAttacker(go1, 1), attackers.StackelbergAttacker(go1, 1)]
go1.set_players(d1, a1, p1)

go2 = game.GameWithObservabilities(values, time_horizon)
go2.attackers = [1]
go2.defenders = [0]
a2 = [attackers.ObservingStackelbergAttacker(go2, 1)]
d2 = [bm.FB(go2, 0)]
p2 = [attackers.ObservingStackelbergAttacker(go2, 1), attackers.StackelbergAttacker(go2, 1)]
go2.set_players(d2, a2, p2)

In [32]:
def print_header(targets, profiles, observabilities):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile," for x in range(len(profiles[0]['others']))]) + ",".join(["Obs" + str(i) for i in range(len(observabilities))]) + "\n" 

In [33]:
def print_row(targets, game_type, time_horizon,d,p,obs):
    return ",".join([str(i) for i in ([d+"_vs_"+p["attacker"]+"_"+game_type] + [time_horizon] + targets + [d] + [p["attacker"]] + [x for x in p["others"]] + obs) ])+"\n"

In [34]:
profiles = []
profiles.append({"attacker": util.print_adv(a[0]), 
                 "others": [util.print_adv(p[0])]
                })
profiles1 = []
profiles1.append({"attacker": util.print_adv(a2[0]), 
                 "others": [util.print_adv(p2[1])]
                 })

In [35]:
(print_header(targets, profiles, obs_vect))

'Name,T,0,1,2,3,4,Defender,Attacker,Profile,Obs0,Obs1,Obs2,Obs3,Obs4\n'

In [36]:
defender =["FB1", "FB1","FB1"]

In [37]:
folder = "../Experiments/Experiment3"
batch_name = "belief_max_obs_vs_no_obs"
batch_file = batch_name + ".csv"
batch_path = folder + "/" + batch_file

In [38]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, profiles, obs_vect))
    f.write(print_row(targets, "no_obs", time_horizon,defender[0],profiles[0],[]))
    f.write(print_row(targets, "obs", time_horizon,defender[1],profiles[0], obs_vect))
    f.write(print_row(targets, "obs", time_horizon,defender[2],profiles1[0], obs_vect))

In [39]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,Defender,Attacker,Profile,Obs0,Obs1,Obs2,Obs3,Obs4
0,FB1_vs_sta1_no_obs,1000,5,3,8,4,9,FB1,sta1,sta1,NaN,NaN,NaN,NaN,NaN
1,FB1_vs_sta1_obs,1000,5,3,8,4,9,FB1,sta1,sta1,0.95,0.95,0.75,0.95,0.95
2,FB1_vs_obsta1_obs,1000,5,3,8,4,9,FB1,obsta1,obsta1,0.95,0.95,0.75,0.95,0.95


In [40]:
b = runner.Batch(batch_path, folder)

In [41]:
b.parse_batch()

In [42]:
batches = []
batches.append(b)

In [43]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

In [44]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

C:\Users\Mattia\Anaconda3\lib\site-packages\matplotlib\ticker.py:2207: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


In [45]:
g = deepcopy(batches[0].configurations[2].game)

In [46]:
print(g)

1000,5.0,3.0,8.0,4.0,9.0,obsta1,FB1,obsta1


In [47]:
isinstance(g, game.GameWithObservabilities)

True

In [48]:
g.players

{1: <ObservingStackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>}

In [49]:
g.observabilities

{0: 0.95, 1: 0.95, 2: 0.75, 3: 0.95, 4: 0.95}

In [50]:
e = runner.Experiment(g)

In [51]:
for i in range(900):
    e.run_interaction_with_observations()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [0, 0, 1